# Sometimes, models need additional evaluation after training is completed...
***

# Prepare Experiment & Deepspeed config (**MANDATORY**)
***

In [2]:
ds_config = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "zero_optimization": {
        "stage": 3,
        "allgather_partitions": True,
        "allgather_bucket_size": 5e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 5e8,
        "contiguous_gradients": True,
        "offload_optimizer": {
            "device": "none",
        },
        "offload_params": {
            "device": "none"
        },
    },
    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 200,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": False
}

training_args = {
    "num_train_epochs": 4,
    "gradient_accumulation_steps": 1,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 256,
    "fp16": True,
    "weight_decay": 0.0,
    "warmup_steps": 0,
    "learning_rate": 1e-5,
    "logging_strategy": "epoch",
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "save_total_limit": 1,
    "load_best_model_at_end": True,
    "metric_for_best_model": "eval_accuracy",
    "greater_is_better": True,
}

# usually overriden by external config:
num_gpus = 1
model_name ="bert-base-uncased"
block_size = 128
logdir = "data/models/bert-base-uncased/anti_ms/"
override_logdir = False
dataset_folder = "data/contrastive_moral_stories/anti_ms/action+norm/norm_distance/"
checkpoint = None

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import pandas as pd
from datasets import load_dataset
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
from transformers import Trainer, TrainingArguments

2022-06-15 01:26:37.598685: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Tokenize the dataset
***

In [ ]:
if "Eleuther" in model_name:
    tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token='<|startoftext|>', 
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def load_action_norm_split(path):
    train, dev, test = [pd.read_json(f"{path}{x}.jsonl", lines=True) for x in ["train", "dev", "test"]]

    # construct dataframes that can actually be used
    assign_action = lambda x: x["moral_action"] if x["label"] == 1 else x["immoral_action"]
    train["action"] = train.apply(assign_action, axis=1)
    dev["action"] = dev.apply(assign_action, axis=1)
    test["action"] = test.apply(assign_action, axis=1)

    subset = ["norm", "action", "label"]
    train = train[subset]
    dev = dev[subset]
    test = test[subset]
    return train, dev, test


train, dev, test = load_action_norm_split("data/contrastive_moral_stories/original_ms/action+norm/norm_distance/")
opt_train, opt_dev, opt_test = load_action_norm_split("data/contrastive_moral_stories/optional_ms/action+norm/norm_distance/")
anti_train, anti_dev, anti_test = load_action_norm_split("data/contrastive_moral_stories/anti_ms/action+norm/norm_distance/")
contra_train, contra_dev, contra_test = load_action_norm_split("data/contrastive_moral_stories/contra_ms/action+norm/norm_distance/")

In [ ]:
dataset = datasets.DatasetDict()

# (N, A_M, 1)
# normal norms, moral actions: test dataset
# (N, A_I, 0)
dataset["original_ms"] = datasets.Dataset.from_pandas(test)

# (ON, A_M, 1)
# optional norms, normal actions: optional dataset
# (ON, A_I, 1)
dataset["optional_ms"] = datasets.Dataset.from_pandas(opt_test)

# (~N, A_M, 0)
# (~N, A_I, 1)
# anti_norms, negated labels
dataset["anti_ms"] =  datasets.Dataset.from_pandas(anti_test)

# everything above stacked
dataset["contra_ms"] =  datasets.Dataset.from_pandas(contra_test)

In [ ]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

def tokenize(samples):
    return tokenizer(samples["action"], samples["norm"], truncation=True, padding="max_length", max_length=128)

tokenized_data = dataset.map(tokenize, batched=True, batch_size=10000)

# Load the model

In [ ]:
if checkpoint is None:
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
else:
    if checkpoint in {"first", "last"}:
        ckpts = [x for x in os.listdir(logdir) if x.startswith("checkpoint")]
        ckpts = sorted(ckpts, key=lambda x: int(x.split("-")[1]))
        ckpt = ckpts[0 if checkpoint == "first" else -1]
        print("ATTEMPTING TO LOAD", os.path.join(logdir, ckpt))
        model = AutoModelForSequenceClassification.from_pretrained(os.path.join(logdir, ckpt))
    elif checkpoint == "no-pretraining":
        # TODO
        pass
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

model.resize_token_embeddings(len(tokenizer))

if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id

# Prepare Trainer
***

In [ ]:
training_args = TrainingArguments(
    output_dir=logdir,
    overwrite_output_dir=False,
    logging_dir=logdir,
    deepspeed= logdir + "/ds_config.json",
    report_to="tensorboard",
    **training_args
)

In [ ]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
)

In [ ]:
for split, data in tokenized_data.items():
    trainer.evaluate(data, metric_key_prefix=split)